# Othogonal Algebra

Use the Affine Functions that map to $2^a$ to create the algebra of Collatz values by generation

## The equations:

- $ V_{1} =  \frac{2^{1}}{3^{1}} - (   \frac{2^{0}}{3^{1}} ) $
- $ V_{10} =  \frac{2^{2}}{3^{1}} - (   \frac{2^{1}}{3^{1}} ) $
- $ V_{01} = \frac{2^{2}}{3^{1}} - (   \frac{2^{0}}{3^{1}} ) $
- $ V_{00} = \frac{2^{2}}{3^{2}} - (   \frac{2^{0}}{3^{1}} + \frac{2^{1}}{3^{2}} ) $

Which translate to linear equations:

- $a=0$
    - $\frac{1}{2^0}x_{\emptyset} = 1; \quad x_{\emptyset} = 1$
- $a=1$
    - $\frac{1}{2^1}x_{1} = 1; \quad x_{1} = 2$
    - $-3x_{1} + x_{0} = 1; \quad x_{0} = \frac{1}{3}$
- $a=2$
    - $\frac{1}{2^2}x_{11} = 1; \quad x_{11} = 4 $
    - $\frac{2^{2}}{3^{1}} - (   \frac{2^{1}}{3^{1}} ) = 1; \quad x_{10} = \frac{2}{3} $
    - $\frac{1}{2^2}x_{11} = 1; \quad x_{11} = 4 $
    - $\frac{1}{2^2}x_{11} = 1; \quad x_{11} = 4 $
 


# Linear Algebra Representation

These equations then become the bordered diagonal matrices of rank $2^a$ like:

$$
\begin{pmatrix} 1&0& \dots &0\\1&-{\alpha}_{1} & \dots &0\\ \dots & \dots & \dots & \dots\\1&0& \dots & -{\alpha}_{n}\end{pmatrix}x = 
\begin{pmatrix} 2^a\\{\beta}_{1}\\ \dots \\{\beta}_{n}\end{pmatrix}
$$
Where:
- the first column is all 1's
- the diagonals below row 0 are the negatives of the $\alpha$ values of the affine transform
- $x_0 = 2^a$
- the rest of the $x$ vector values are the $\beta$ values of the affine transforms

Thus we can trivially compute the values of all nodes (the $x_i$) in a generation directly using this form.

In [1]:
import numpy as np
import math
import io
import pandas as pd
from fractions import Fraction
from kenglish.mrlattice import (
    countZeros,
    mrTupFromPath,
    mrTupValue,
    generationAffineParamsFromPath,
    mrTupToLaTex,
    write_generationIntegersForwardWithMeta
)
from scipy.sparse import coo_array
from scipy.sparse.linalg import spsolve


In [2]:
def generationAffineParamsFromLabelDirect(label):
    """
    This is a much simpler routine that uses the label to directly drive the
    computation of the $\alpha x + \beta$ affine transform the label to $2^{len(label)}$
    """
    zeros = [i for i, bit in enumerate(label) if bit == '0']
    b = len(zeros)
    alpha = 3**b
    beta = 0
    d = b - 1
    for c in zeros:
        beta += ((2**c)*(3**d))
        d -= 1
    return (alpha, beta)
#

In [6]:
def generationAffineParamsFromLabelDirect(label):
    """
    This is a much simpler routine that uses the label to directly drive the
    computation of the $\alpha x + \beta$ affine transform the label to $2^{len(label)}$
    """
    zeros = [idx for idx, bit in enumerate(label) if bit == '0']
    b = len(zeros)
    alpha = 3**b
    beta = 0
    for i in range(len(zeros)):
        c = zeros[i]
        d = b - i - 1
        beta += ((2**c)*(3**d))
    return (alpha, beta)
#

In [7]:
generationAffineParamsFromLabelDirect("011")

(3, 1)

In [8]:
generationAffineParamsFromLabelDirect("10")

(3, 2)

In [9]:
generationAffineParamsFromLabelDirect("00")

(9, 5)

In [10]:
def generationIntegersDirect(a):
    """
    Generate all integers in the given generation $a$ using only the label of 
    each lattice node to drive the math.

    Due to the sparcity of the integers in the graph, it is faster to
    forward generate, than do it this way.  But this shows
    the math works.
    """
    edge_val = 2**a
    for i in range(edge_val):
        if i == 0:
            yield edge_val
        else:
            idx = 2**a - (i+1)
            label = f"{idx:0{a}b}"
            alpha, beta = generationAffineParamsFromLabelDirect(label)
            delta = edge_val - beta
            if delta % alpha == 0:
                yield delta // alpha
#


In [11]:
list(generationIntegersDirect(4))

[16, 4, 5, 1]

In [12]:
list(generationIntegersDirect(5))

[32, 8, 10, 2, 3]

In [13]:
def countYields(yielder):
    n = 0
    for _ in yielder:
        n += 1
    return n
#

In [15]:
for a in range(3, 16, 1):
    print((a, countYields(generationIntegersDirect(a))))

(3, 2)
(4, 4)
(5, 5)
(6, 8)
(7, 10)
(8, 14)
(9, 18)
(10, 26)
(11, 36)
(12, 50)
(13, 67)
(14, 89)
(15, 117)


In [21]:
label = "10"
T = mrTupFromPath(label)
print(mrTupValue(T))
print(generationAffineParamsFromPath(label)
mrTupToLaTex(T)

(2, 3)
(3, 2)


'$ \\frac{2^{2}}{3^{1}} - (   \\frac{2^{1}}{3^{1}} ) $'

In [22]:
label = "01"
T = mrTupFromPath(label)
print(mrTupValue(T))
print(generationAffineParamsFromPath(label))
mrTupToLaTex(T)

(1, 1)
(3, 1)


'$ \\frac{2^{2}}{3^{1}} - (   \\frac{2^{0}}{3^{1}} ) $'

In [23]:
label = "00"
T = mrTupFromPath(label)
print(mrTupValue(T))
print(generationAffineParamsFromPath(label))
mrTupToLaTex(T)

(-1, 9)
(9, 5)


'$ \\frac{2^{2}}{3^{2}} - (   \\frac{2^{0}}{3^{1}} + \\frac{2^{1}}{3^{2}} ) $'

### The equations  can then be aggregated into a bordered diagonal matrix:

#### Generation 1:


In [26]:
A = [[ 1.0,  0.0], 
     [ 1.0, -3.0]]
Y = [2, 1]
np.linalg.solve(A, Y)

array([2.        , 0.33333333])

#### Generation 2:

In [25]:
A = [[1.0,  0.0,  0.0,  0.0], 
     [1.0, -3.0,  0.0,  0.0],
     [1.0,  0.0, -3.0,  0.0],
     [1.0,  0.0,  0.0, -9.0],
    ]
Y = [4, 2, 1, 5]
np.linalg.solve(A, Y)

array([ 4.        ,  0.66666667,  1.        , -0.11111111])

In [17]:
def makeGenerationAlgebra_Ay(a):
    A_vals = []
    A_row = []
    A_col = []
    y = np.zeros((2**a))
    # Left most column is always 1
    for i in range(2**a):
        A_vals.append(1)
        A_row.append(i)
        A_col.append(0)
        if i == 0:
            y[0] = 2**a
        else:
            idx = 2**a - (i+1)
            label = f"{idx:0{a}b}"
            alpha, beta = generationAffineParamsFromLabelDirect(label)
            y[i] = beta
            A_vals.append(-alpha)
            A_row.append(i)
            A_col.append(i)
    return (coo_array((np.array(A_vals), (np.array(A_row), np.array(A_col))), shape=(2**a, 2**a))).tocsr(), y

In [18]:
a = 15
A, Y = makeGenerationAlgebra_Ay(a)
X = spsolve(A, (3**a)*Y)
for x in X:
    f = Fraction(int(x), 3**a)
    d = f.denominator
    if d == 1:
        n = f.numerator
        print(a, n)
a = a+1
A, Y = makeGenerationAlgebra_Ay(a)
X = spsolve(A, (3**a)*Y)
for x in X:
    f = Fraction(int(x), 3**a)
    d = f.denominator
    if d == 1:
        n = f.numerator
        print(a, n)

15 32768
15 8192
15 10240
15 2048
15 3072
15 10752
15 2560
15 512
15 3328
15 768
15 10880
15 2688
15 640
15 128
15 3392
15 832
15 1088
15 192
15 10912
15 2720
15 672
15 160
15 3616
15 32
15 1120
15 352
15 3408
15 848
15 1104
15 208
15 272
15 3632
15 48
15 1200
15 368
15 112
15 10920
15 2728
15 680
15 168
15 3624
15 40
15 1128
15 360
15 904
15 8
15 280
15 88
15 1208
15 120
15 3412
15 852
15 1108
15 212
15 276
15 3636
15 52
15 1204
15 372
15 116
15 68
15 36
15 908
15 12
15 300
15 92
15 28
15 10922
15 2730
15 682
15 170
15 3626
15 42
15 1130
15 362
15 906
15 10
15 282
15 90
15 1210
15 122
15 226
15 2
15 402
15 70
15 38
15 22
15 302
15 30
15 3413
15 853
15 1109
15 213
15 277
15 3637
15 53
15 1205
15 373
15 117
15 69
15 37
15 909
15 13
15 301
15 93
15 29
15 1137
15 369
15 17
15 401
15 9
15 227
15 3
15 403
15 75
15 23
15 7
16 65536
16 16384
16 20480
16 4096
16 6144
16 21504
16 5120
16 1024
16 6656
16 1536
16 21760
16 5376
16 1280
16 256
16 6784
16 1664
16 2176
16 384
16 21824
16 5440
16 1344

In [19]:
A

<Compressed Sparse Row sparse array of dtype 'int64'
	with 131071 stored elements and shape (65536, 65536)>